# Recursive data and code
---

> The power of recursion evidently lies in the possibility of defining an infinite set of objects by a finite statement. In the same manner, an infinite number of computations can be described by a finite recursive program, ....
>     — Niklaus Wirth, Algorithms + Data Structures = Programs

Read the quote _three_ times. And keep it in the back of your mind while reading this notebook.

A recursive definition (data or code) has _one or more base cases_, meaning input(s) for which a result is produced trivially (without recurring), and _one or more recursive cases_, meaning input(s) for which the program recurs (calls itself).

### Recursive data
---
Scala allows us to define potentially infinite data (but finite at the run-time) using recursion. As described, we need a base case and recursive case to define one. The common data structure in Function Programming, and a good example as a recursive data structure is, the linked list. 

A linked list is a collection of elements, where each element is linked to the next, and at the end holds a sentinel value to indicate the end of the list. In other words, a _list_ is a _pair_ or an _end_, where pair has an element and list, and end is an indicator of end of the list. As you have noticed, the last statement is described in the terms of Algebraic Data Type, we can represent it in Scala as follows.

In [1]:
sealed trait IntList
final case class Pair(element: Int, rest: IntList) extends IntList
final case object End extends IntList

defined trait IntList
defined class Pair
defined object End

Let's create a list of elements from 1 to 3.

In [2]:
// [1, [2, [3, End]]]

val end: IntList = End
val three_pair: IntList = Pair(3, end)
val two_pair: IntList = Pair(2, three_pair)
val list: IntList = Pair(1, two_pair)

end: IntList = End
three_pair: IntList = Pair(element = 3, rest = End)
two_pair: IntList = Pair(element = 2, rest = Pair(element = 3, rest = End))
list: IntList = Pair(
  element = 1,
  rest = Pair(element = 2, rest = Pair(element = 3, rest = End))
)

In [3]:
// Or

val list: IntList = Pair(1, Pair(2, Pair(3, End)))

list: IntList = Pair(
  element = 1,
  rest = Pair(element = 2, rest = Pair(element = 3, rest = End))
)

Let's look at the importance of base case,

In `IntList`, the `End` is the base case and that the reason we can create an instance of the linked list. What would happen if we don't add base case?
Let's define `Broken` without any base case.

In [4]:
final case class Broken(broken: Broken)

defined class Broken

The Scala compiler would allow us to define such "broken" recursive structure, but _we won't be able to create an instance_ of it. The reason is, to create an instance of `Broken` we need an instance of `Broken`, which obviously we don't have because we are trying to create one in the first place. 😅

By following the same reasoning, if we don't add `End` (the base case) in linked list data structure then we won't be able to create an instance of linked list.

Let's re-read the first part of the quote,

> The power of recursion evidently lies in the possibility of defining an infinite set of objects by a finite statement.

Now it would be apparent that using _finte set of declarations_ (`End` and `Pair`), we are able to create an instance of list with "theoretically" _infinite elements_ (objects).

Let's see one more example before jumping onto implementing operations on these recursive data structures. The second common data structure is called Tree, and having data represented as a tree makes some operation easy on them. A _tree_ is a _node_ or _leaf_. A node has a left branch and right branch, which are the tree itself, and a left holds a value. Let's define it using ADT,

In [5]:
sealed trait StringTree
final case class Node(left: StringTree, right: StringTree) extends StringTree
final case class Leaf(value: String) extends StringTree

defined trait StringTree
defined class Node
defined class Leaf

### Recursive code
---

The most natural way to define an operation on recursive data structure is using recursion. We will apply the same principle of base case(s) and recursive case(s) to make sure that recursive code doesn't "end" somewhere. Let's see a common example of a recursive method, a factorial operation.

In [6]:
// 3! = 3 x 2 x 1 = 6

def factorial(n: Int): Int = 
    if (n == 0) 1               // base case
    else n * factorial(n - 1)   // recursive case

val result = factorial(3)       // 6

defined function factorial
result: Int = 6

If we don't add the base case in the above method, the method will don't know when to "stop" the computation and will keep calling itself subtracting one from the input `n`. In a recursive data structure without a base case, we won't be able to create an instance of it. Same way, in recursive code, we won't get "returned" from a recursive method where the base case is not defined. And because the stack memory is limited, Scala will raise the `StackOverflowError` exception.

In [6]:
/**
def brokenFactorial(n: Int): Int =
    n * brokenFactorial(n - 1)

brokenFactorial(3)

java.lang.StackOverflowError
  ammonite.$sess.cmd11$Helper.brokenFactorial(cmd11.sc:8)
  ammonite.$sess.cmd11$Helper.brokenFactorial(cmd11.sc:8)
*/

As we discussed in the previous notebook, we can use `polymorephism` or `pattern matching` to write an operation ADT. Let's stick to the pattern matching, as it's more used in functional programming. Let's write a couple of common operations on a linked list using recursion.

1. sum, to get back the summation of all the elements in the list
    - to sum a list of elements, you need to add a first element with the sum of rest of the list.
2. length, to get the length of list
    - to compute the length, add one to the length of the rest of the list.
    
As you can see, the problem (sum or length) is expressed by dividing the list (first, rest) and recursively solving the sub-problem (rest). You can see the "recursion" in the solution statement itself.

In [7]:
object IntListRecur {
    
    def sum(list: IntList): Int = list match {
        case End => 0
        case Pair(first, rest) => first + sum(rest)
    }
    
    def length(list: IntList): Int = list match {
        case End => 0
        case Pair(_, rest) => 1 + length(rest)
    }
    
}

// we using `list` defined above, [1,2,3]
val sum_list = IntListRecur.sum(list)          // 6
val length_list = IntListRecur.length(list)    // 3

defined object IntListRecur
sum_list: Int = 6
length_list: Int = 3

Now, re-read the second part of the quote,

> In the same manner, an infinite number of computations can be described by a finite recursive program

The two lines of code (finite program) can perform infinite steps of computation, `plus` for sum and `add 1` for length. But this ease of expressing computation comes with a cost, because of the finiteness of memory. If you create a list of millions of millions of elements, and call sum recursively, it will end up throwing the `StackOverflowError` exception. 

In [8]:
object IntList {
    
    // this is helper method to create IntList from variable legnth arguments
    // and to illustrate that recursive methods on large input may fail with
    // `StackOverflowError` exception.

    // `Int*` means variable length argument
    // one can pass any number of integer arguments to the method
    // inside the method, `is` is Seq[Int], that's why we are converting it to List using `.toList` to pattern match on it
    // `::` is like our `Pair` for Scala's List data structure
    // `_*` expands list to pass into variable length argument
    def apply(is: Int*): IntList = is.toList match {
        case Nil => End
        case head :: tail => Pair(head, apply(tail: _*))
    }
    
}

IntList(1, 2, 3) // same as previously defined `list` value above.

/**
// `1 to 10000` creates range of elements from 1 to 10000.

IntList(1 to 10000: _*)

java.lang.StackOverflowError
  ammonite.$sess.cmd0$.instance(cmd0.sc:7)
  ammonite.$sess.cmd7$Helper$IntList$.apply(cmd7.sc:5)
*/

defined object IntList
res7_1: IntList = Pair(
  element = 1,
  rest = Pair(element = 2, rest = Pair(element = 3, rest = End))
)

### tail recursive
---

To overcome such situation where we are making too much recursive calls, we can rewrite our recursive method in such a way that Scala compiler can compile it to iterative-style, meaning rather than allocating new stack for each method call, it uses same stack to perform the recursive operation. Due to limitations of JVM, Scala only supports Tail Call Optimization (TCO) on direct recursion.

But there is this condition that should be matched. A recursive call should be the final call of the method, meaning it should not perform any other operation than calling itself. For example, in above `sum` and `length` methods, respected recursive calls perform `first +` and `1 +`. It means, they don't satisfy the condition we need to convert a recursive call to an iterative one.

To remove extra operation on recursive calls of those methods, we can add an `accumulator` parameter to hold "sum" or "length" of elements we have already visited. Let's implement them: 

In [9]:
object IntListIter {
    
    // we are defining method within method so consumer don't need to provide an accumulator herself.
    
    def sum(list: IntList): Int = {
        def loop(list: IntList, acc: Int): Int = list match {
            case End => acc
            case Pair(first, rest) => loop(rest, first + acc)
        }
        loop(list, 0)
    }
    
    def length(list: IntList): Int = {
        def loop(list: IntList, acc: Int): Int = list match {
            case End => 0
            case Pair(_, rest) => 1 + length(rest)
        }
        loop(list, 0)
    }
    
}

IntListIter.sum(list)
IntListIter.length(list)

defined object IntListIter
res8_1: Int = 6
res8_2: Int = 3

In [10]:
object IntList {
    
    // the code is just to illustrate that tail recursive method would not throw `StackOverflowError` exception.
    // try to understand what's going on..
    
    def apply(is: Int*): IntList = {
        def loop(ls: List[Int], acc: IntList): IntList = ls match {
            case Nil => acc
            case head :: tail => loop(tail, Pair(head, acc))
        }
        loop(is.toList.reverse, End)
    }
    
}

IntList(1 to 10000: _*) // would not throw `StackOverflowError` exception.

defined object IntList
res9_1: IntList = Pair(
  element = 1,
  rest = Pair(
    element = 2,
    rest = Pair(
      element = 3,
      rest = Pair(
        element = 4,
        rest = Pair(
          element = 5,
          rest = Pair(
            element = 6,
            rest = Pair(
              element = 7,
              rest = Pair(
                element = 8,
                rest = Pair(
                  element = 9,
                  rest = Pair(
                    element = 10,
                    rest = Pair(
                      element = 11,
                      rest = Pair(
                        element = 12,
                        rest = Pair(
                          element = 13,
                          rest = Pair(
                            element = 14,
                            rest = Pair(
                              element = 15,
                              rest = Pair(
                                element = 16,
                               

Lastly, there is an annotation to let the compiler know that we expect a method to compile in iterative-style. If it's not possible then the compiler will raise an error.

In [11]:
import scala.annotation.tailrec

def sumIter(list: IntList): Int = {
    @tailrec
    def loop(list: IntList, acc: Int): Int = list match {
        case End => acc
        case Pair(first, rest) => loop(rest, first + acc)
    }
    loop(list, 0)
}

import scala.annotation.tailrec


defined function sumIter

As you can see, we added `@tailrec` annotation above the `loop` method and Scala does compile the code which means it will convert this recursive method into iterative-style. But let's try to add the same annotation on a non-tail-recursive method, as we defined earlier. It will not compile the code and explain that "a recursive call not in tail position".

In [11]:
@tailrec
def sum(list: IntList): Int = list match {
    case End => 0
    case Pair(first, rest) => first + sum(rest)
}

cmd11.sc:2: could not optimize @tailrec annotated method sum: it contains a recursive call not in tail position
def sum(list: IntList): Int = list match {
                              ^Compilation Failed

: 

### Exercise
---

1. We created the `StringTree` ADT to represent tree data structure. Add one more case to represent a part where the node does not have a leaf, let's name it `Nil`.

2. Write a method `size` that counts the number of nodes (leaves and branches) in a tree.

3. Write a method to concatenate two IntList, name it `concat`.

4. Write a method to multiply elements in IntList, name it `product`. first using plain recursion and another using tail-recursion. Don't forget to add `@tailrec` annotation on the later one.

5. Write an ADT for mathematical expression containing addition and subtraction operation. Also, write `eval` method to evaluate a given expression. [hint: ADT will have three cases, 2 for operations and 1 for number]